In [7]:
from operator import itemgetter
from random import shuffle


csv_header = 'tail_number,origin,destination,departure_time,arrival_time'
file_name = 'flight_schedule.csv'


def Minutes_since_midnight(time):
    time = int(time)
    return (time // 100) * 60 + (time % 100)


def Military_time(time):
    return (time // 60) * 100 + time % 60


def CalculateGroundTime(origin):
    if origin == 'AUS':
        return 25
    elif origin == 'DAL':
        return 30
    elif origin == 'HOU':
        return 35

    
def print_flight_schedule(fn, csv_hdr, flt_sched):
    with open(fn, 'wt') as f:
        print(csv_hdr, file=f)
        for s in flt_sched:
            print(','.join(s), file=f)


flight_times = [['AUS', 'DAL', 50],['DAL', 'HOU', 65], ['AUS', 'HOU', 45]]


def fly_time(origin, dest):
    fly_time = 0
    for lst in flight_times:
        if (lst[0] == origin and lst[1] == dest) or (lst[1] == origin and lst[0] == dest):
            fly_time = lst[2]
            break;
    return fly_time


class Airport_Status:
    def __init__(self, airportName, noOfGates, gateStatus):
        self.airportName = airportName
        self.noOfGates = noOfGates
        self.gateStatus = gateStatus


class Flight_Status:
    def __init__(self, tailNumber, currentAirport, time, updated):
        self.tailNumber = tailNumber
        self.currentAirport = currentAirport
        self.time = time
        self.updated = updated


def check_Availability(origin, destination, time, Airports):
    time = int(time)
    i = 1
    value = False
    time_available = Military_time(fly_time(origin, destination) + Minutes_since_midnight(time))

    for port in Airports:
        if port.airportName == destination:
            for status in port.gateStatus:
                if int(status[0]) <= time_available and int(status[1]) >= time_available:
                    
                    value = True
                    break
                else:
                    i += 1
                    value = False
    return value, i, str(time_available).rjust(4, '0')


def update_Airport_Status(airport, i, fly_time):
    value = Military_time(Minutes_since_midnight(int(fly_time)) + \
                         CalculateGroundTime(airport.airportName) + 5)
    if value < 2200:
        airport.gateStatus[i - 1][0] = str(value).rjust(4, '0')
    else:
        pass


def update_current_flight_status(flight, airportName, fly_time):
    flight.currentAirport = airportName
    flight.time = str(Military_time(Minutes_since_midnight(int(fly_time)) + \
                               CalculateGroundTime(airportName))).rjust(4, '0')
    flight.updated = True


def update_leftout_flight(flight):
    value = Military_time(Minutes_since_midnight(int(flight.time)) + \
                         2 * CalculateGroundTime(flight.currentAirport))
    flight.time = str(value).rjust(4, '0')
    flight.updated = True


def check_allflight_status(Flights):
    for x in Flights:
  
        if int(x.time) >= 2200:
            return False
            break
        else:
            return True


def lineup_flights(Flights, Airports):
    count = dict.fromkeys(['AUS', 'DAL', 'HOU'], 0)
    flt_last_schedule = list()
    
    for ap in Airports:
   
        for flight in Flights:
            
            if ap.airportName == flight.currentAirport:
                count[ap.airportName] = count.get(flight.currentAirport, 0) + 1
               
    if count['AUS'] > 1:

        for x in range(count['AUS'] - 1):
            fs_last = list()
      
            if count['HOU'] < 3:
                for flight in Flights:
                    if flight.currentAirport == 'AUS' and flight.updated is True:
                        departureTime = str(Military_time(Minutes_since_midnight(flight.time) + \
                                                         CalculateGroundTime('AUS'))).rjust(4, '0')
                        arrivalTime = str(Military_time(Minutes_since_midnight(departureTime) + \
                                                       fly_time('AUS', 'HOU'))).rjust(4, '0')
                        fs_last.extend((flight.tailNumber, flight.currentAirport, 'HOU', departureTime, arrivalTime))
                      
                        count['HOU'] += 1
                        count['AUS'] -= 1
                        flight.updated = False
                        break
                if len(fs_last) != 0: flt_last_schedule.append(fs_last)
            else:
                for flight in Flights:
                    if flight.currentAirport == 'AUS' and flight.updated is True:
                        departureTime = str(Military_time(Minutes_since_midnight(flight.time) + \
                                                         CalculateGroundTime('AUS'))).rjust(4, '0')
                        arrivalTime = str(Military_time(Minutes_since_midnight(departureTime) + \
                                                       fly_time('AUS', 'DAL'))).rjust(4, '0')
                        fs_last.extend((flight.tailNumber, flight.currentAirport, 'DAL', departureTime, arrivalTime))
                    
                        count['DAL'] += 1
                        count['AUS'] -= 1
                        flight.updated = False
                        break
                if len(fs_last) != 0: flt_last_schedule.append(fs_last)

    if count['DAL'] > 2:
     
        for x in range(count['DAL'] - 2):
            fs_last = list()

            if count['HOU'] < 3:
                for flight in Flights:
                    if flight.currentAirport == 'DAL' and flight.updated is True:
                        departureTime = str(Military_time(Minutes_since_midnight(flight.time) + \
                                                         CalculateGroundTime('DAL'))).rjust(4, '0')
                        arrivalTime = str(Military_time(Minutes_since_midnight(departureTime) + \
                                                       fly_time('DAL', 'HOU'))).rjust(4, '0')
                        fs_last.extend((flight.tailNumber, flight.currentAirport, 'HOU', departureTime, arrivalTime))
             
                        count['HOU'] += 1
                        count['DAL'] -= 1
                        flight.updated = False
                        break
                if len(fs_last) != 0: flt_last_schedule.append(fs_last)
            else:
                for flight in Flights:
                    if flight.currentAirport == 'DAL' and flight.updated is True:
                        departureTime = str(Military_time(Minutes_since_midnight(flight.time) + \
                                                         CalculateGroundTime('DAL'))).rjust(4, '0')
                        arrivalTime = str(Military_time(Minutes_since_midnight(departureTime) + \
                                                       fly_time('DAL', 'AUS'))).rjust(4, '0')
                        fs_last.extend((flight.tailNumber, flight.currentAirport, 'AUS', departureTime, arrivalTime))
           
                        count['AUS'] += 1
                        count['DAL'] -= 1
                        flight.updated = False
                        break
                if len(fs_last) != 0: flt_last_schedule.append(fs_last)

    if count['HOU'] > 3:
       
        for x in range(count['HOU'] - 3):
            fs_last = list()
    
            if count['DAL'] < 2:
                for flight in Flights:
                    if flight.currentAirport == 'HOU' and flight.updated is True:
                        departureTime = str(Military_time(Minutes_since_midnight(flight.time) + \
                                                         CalculateGroundTime('HOU'))).rjust(4, '0')
                        arrivalTime = str(Military_time(Minutes_since_midnight(departureTime) + \
                                                       fly_time('HOU', 'DAL'))).rjust(4, '0')
                        fs_last.extend((flight.tailNumber, flight.currentAirport, 'DAL', departureTime, arrivalTime))
                
                        count['DAL'] += 1
                        count['HOU'] -= 1
                        flight.updated = False
                        break
                if len(fs_last) != 0: flt_last_schedule.append(fs_last)
            else:
                for flight in Flights:
                    if flight.currentAirport == 'HOU' and flight.updated is True:
                        departureTime = str(Military_time(Minutes_since_midnight(flight.time) + \
                                                         CalculateGroundTime('HOU'))).rjust(4, '0')
                        arrivalTime = str(Military_time(Minutes_since_midnight(departureTime) + \
                                                       fly_time('HOU', 'AUS'))).rjust(4, '0')
                        fs_last.extend((flight.tailNumber, flight.currentAirport, 'AUS', departureTime, arrivalTime))
                 
                        count['AUS'] += 1
                        count['HOU'] -= 1
                        flight.updated = False
                        break
                if len(fs_last) != 0: flt_last_schedule.append(fs_last)

    return flt_last_schedule


def update_last_flightStatus(flt_schedule, Flights):
    for flight in Flights:
        flight.updated = False

        for fs in flt_schedule:
            if fs[0] == flight.tailNumber and flight.updated is False:
                flight.updated = True
                flight.currentAirport = fs[2]
                flight.time = fs[4]
                break
            else:
                continue
            
AUS = Airport_Status('AUS', 1, [['0601', '2000']])
DAL = Airport_Status('DAL', 2, [['0601', '2000'], ['0601', '2000']])
HOU = Airport_Status('HOU', 3, [['0601', '2000'], ['0601', '2000'], ['0601', '2000']])

Airports = [AUS, DAL, HOU]

t1 = Flight_Status('T1', 'AUS', '0600', True)
t2 = Flight_Status('T2', 'DAL', '0600', True)
t3 = Flight_Status('T3', 'DAL', '0600', True)
t4 = Flight_Status('T4', 'HOU', '0600', True)
t5 = Flight_Status('T5', 'HOU', '0600', True)
t6 = Flight_Status('T6', 'HOU', '0600', True)

Flights = [t1, t2, t3, t4, t5, t6]

flt_schedule = list()


def create_flight_schedule(Flights, Airports):
    
    while check_allflight_status(Flights):
        shuffle(Airports) 
        for flight in Flights:
            fs = list()
            flight.updated = False
            for airport in Airports:
                if flight.currentAirport != airport.airportName:
                    (value, i, fly_time) = check_Availability(flight.currentAirport, airport.airportName, flight.time, Airports)
                    if value is True:
                        fs.extend((flight.tailNumber, flight.currentAirport, airport.airportName, flight.time, fly_time))
                        update_Airport_Status(airport, i, fly_time)
                        update_current_flight_status(flight, airport.airportName, fly_time)
                        break            
            if len(fs) != 0 and int(flight.time) <= 2200: flt_schedule.append(fs)  
            if flight.updated is False: update_leftout_flight(flight)
                
    update_last_flightStatus(flt_schedule[::-1], Flights)
    flt_last_schedule = lineup_flights(Flights, Airports)
    for x in flt_last_schedule:
        
        flt_schedule.append(x) 
        
    flt_schedule.sort(key=itemgetter(0, 3))
    print_flight_schedule(file_name, csv_header, flt_schedule)


create_flight_schedule(Flights, Airports)